## All imports


In [ ]:
import keras
from keras.layers import Rescaling

import numpy as np
from numpy.random import seed
import tensorflow as tf

import pandas as pd
import os
import shutil 

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, MaxPooling2D, BatchNormalization 

from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam

## Set up the dataset

Set kaggle parameters


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'paularodriguezj'  # Your Kaggle username
os.environ['KAGGLE_KEY'] = 'ba6f59a7287fc740d60ae6a035ea6461'  # Your Kaggle API key
os.environ['URN'] = '6650398'  # Your URN: submissions without a URN will not count

Download dataset

In [ ]:
!python3 /usr/local/bin/kaggle competitions download -c uos-com2028-21-22-cw --force
!unzip uos-com2028-21-22-cw.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: train/145000.jpg        
  inflating: train/145001.jpg        
  inflating: train/145002.jpg        
  inflating: train/145003.jpg        
  inflating: train/145004.jpg        
  inflating: train/145005.jpg        
  inflating: train/145006.jpg        
  inflating: train/145007.jpg        
  inflating: train/145008.jpg        
  inflating: train/145009.jpg        
  inflating: train/145010.jpg        
  inflating: train/145011.jpg        
  inflating: train/145012.jpg        
  inflating: train/145013.jpg        
  inflating: train/145014.jpg        
  inflating: train/145015.jpg        
  inflating: train/145016.jpg        
  inflating: train/145017.jpg        
  inflating: train/145018.jpg        
  inflating: train/145019.jpg        
  inflating: train/145020.jpg        
  inflating: train/145021.jpg        
  inflating: train/145022.jpg        
  inflating: train/145023.jpg        
  inflating: train/145024.j

Get the data into diferent folders:

In [ ]:
#create folders to sort pictures into.
!mkdir -p CellsTrain/0
!mkdir -p CellsTrain/1
!mkdir -p CellsTrain/2
!mkdir -p CellsTrain/3
!mkdir -p CellsTrain/4
!mkdir -p CellsTrain/5
!mkdir -p CellsTrain/6
!mkdir -p CellsTrain/7

In [ ]:
#sort the images into their meant folder depending on the cell type.
training_cells = pd.read_csv('train.csv')

for index, row in training_cells.iterrows():
    id1 = row['Id']
    id= str(id1)
    cell_type = row['Cell type']

    if len(id) == 6 : #format IS THE SAME as the foto
        file_name = id + ".jpg"
        shutil.move(f"train/{file_name}",f"CellsTrain/{cell_type}")
    else:
        while len(id) != 6:
            id = "0"+ id
        if len(id) == 6:
            file_name = id + ".jpg"
            shutil.move(f"train/{file_name}",f"CellsTrain/{cell_type}")

##Load the dataset:


In [ ]:
colour_mode = 'grayscale'

# Up to you: change the image size if desired. Perhaps they don't need to be that big.
image_size = (28, 28)

# Up to you: training batch size.
batch_size = 64

train_cells_dataset = image_dataset_from_directory(
  'CellsTrain',
  color_mode=colour_mode,
  validation_split=0.2,
  subset='training',
  seed=42,
  label_mode='categorical',  # There are two classes for this task.
  image_size=image_size,
  batch_size=batch_size)

validation_cells_dataset = image_dataset_from_directory(
  'CellsTrain',
  color_mode=colour_mode,
  validation_split=0.2,
  subset='validation',
  seed=42,
  label_mode='categorical',  # There are two classes for this task.
  image_size=image_size,
  batch_size=batch_size)

test_cells_dataset = image_dataset_from_directory(
  'test',
  shuffle=False,  # Important: the test dataset order must match Kaggle!
  labels=None,  # Important: the test dataset has no labels...
  color_mode=colour_mode,
  image_size=image_size,
  batch_size=1)  # Don't batch the test samples.

print('Train label names:', train_cells_dataset.class_names)
print('Test label names (empty):', test_cells_dataset.class_names)

Found 150000 files belonging to 8 classes.
Using 120000 files for training.
Found 150000 files belonging to 8 classes.
Using 30000 files for validation.
Found 50000 files belonging to 1 classes.
Train label names: ['0', '1', '2', '3', '4', '5', '6', '7']
Test label names (empty): ['']


##Build model

In [ ]:
model = Sequential([
  Rescaling(1./255),

  Conv2D(32, (3,3), padding='same', activation="relu", input_shape=(28,28,1)),
  Conv2D(32, (3,3), padding='same', activation="relu"),
  Conv2D(32, (5,5), padding='same', activation="relu"),
  BatchNormalization(),
  MaxPooling2D(2,2, padding='same'),
  Dropout(0.4),

  Conv2D(64, (3,3), padding='same', activation="relu"),
  Conv2D(64, (3,3), padding='same', activation="relu"),
  Conv2D(64, (5,5), padding='same', activation="relu"),
  BatchNormalization(),
  MaxPooling2D(2,2, padding='same'),
  Dropout(0.4),

  Conv2D(128, (3,3), padding='same', activation="relu"),
  Conv2D(128, (3,3), padding='same', activation="relu"),
  Conv2D(128, (5,5), padding='same', activation="relu"),
  BatchNormalization(),
  MaxPooling2D(2,2, padding='same'),
  Dropout(0.4),

  Flatten(),
  Dense(1024,activation="relu"),
  Dense(1024,activation="relu"),
  Dense(8, activation="softmax")
])

model.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(train_cells_dataset, validation_data=validation_cells_dataset, epochs=60, callbacks=[model_checkpoint_callback])
model.load_weights(checkpoint_filepath)

Epoch 1/60
1875/1875 [==============================] - 86s 40ms/step - loss: 1.3563 - accuracy: 0.4984 - val_loss: 1.2480 - val_accuracy: 0.5402
Epoch 2/60
1875/1875 [==============================] - 100s 53ms/step - loss: 1.2156 - accuracy: 0.5458 - val_loss: 1.3190 - val_accuracy: 0.5111
Epoch 3/60
1875/1875 [==============================] - 96s 51ms/step - loss: 1.1597 - accuracy: 0.5699 - val_loss: 1.1333 - val_accuracy: 0.5805
Epoch 4/60
1875/1875 [==============================] - 87s 46ms/step - loss: 1.1184 - accuracy: 0.5878 - val_loss: 1.2891 - val_accuracy: 0.5604
Epoch 5/60
1875/1875 [==============================] - 93s 49ms/step - loss: 1.0863 - accuracy: 0.5999 - val_loss: 1.0615 - val_accuracy: 0.6101
Epoch 6/60
1875/1875 [==============================] - 91s 48ms/step - loss: 1.0623 - accuracy: 0.6081 - val_loss: 1.0910 - val_accuracy: 0.5997
Epoch 7/60
1875/1875 [==============================] - 93s 50ms/step - loss: 1.0392 - accuracy: 0.6155 - val_loss: 1.0446 

##Set predictions


In [ ]:
predictions = [train_cells_dataset.class_names[np.argmax(x)] for x in model.predict(test_cells_dataset, verbose=1)]
predictions

50000/50000 [==============================] - 215s 4ms/step


['6',
 '6',
 '7',
 '0',
 '6',
 '7',
 '0',
 '6',
 '0',
 '0',
 '6',
 '1',
 '3',
 '4',
 '3',
 '6',
 '0',
 '6',
 '6',
 '7',
 '3',
 '0',
 '7',
 '0',
 '4',
 '3',
 '5',
 '2',
 '1',
 '5',
 '0',
 '0',
 '6',
 '0',
 '0',
 '6',
 '0',
 '0',
 '6',
 '6',
 '4',
 '0',
 '6',
 '3',
 '7',
 '0',
 '7',
 '6',
 '7',
 '0',
 '6',
 '0',
 '6',
 '5',
 '6',
 '6',
 '0',
 '6',
 '0',
 '7',
 '6',
 '3',
 '6',
 '3',
 '3',
 '6',
 '0',
 '0',
 '0',
 '0',
 '3',
 '0',
 '2',
 '0',
 '0',
 '0',
 '6',
 '6',
 '0',
 '3',
 '3',
 '0',
 '6',
 '0',
 '6',
 '3',
 '6',
 '7',
 '2',
 '4',
 '1',
 '6',
 '3',
 '0',
 '0',
 '6',
 '6',
 '6',
 '0',
 '0',
 '4',
 '2',
 '0',
 '0',
 '6',
 '6',
 '7',
 '6',
 '1',
 '6',
 '6',
 '0',
 '0',
 '7',
 '0',
 '0',
 '0',
 '3',
 '4',
 '3',
 '0',
 '3',
 '4',
 '0',
 '0',
 '7',
 '7',
 '0',
 '4',
 '6',
 '6',
 '2',
 '0',
 '3',
 '7',
 '0',
 '0',
 '3',
 '6',
 '3',
 '6',
 '7',
 '6',
 '6',
 '0',
 '0',
 '0',
 '0',
 '3',
 '5',
 '6',
 '3',
 '6',
 '0',
 '6',
 '0',
 '7',
 '7',
 '0',
 '0',
 '7',
 '6',
 '3',
 '7',
 '0',
 '1',
 '6'

In [ ]:
from pandas import DataFrame
DataFrame(data={'Cell Type': predictions}).to_csv('predictions.csv', index_label='Id')

Submit predictions to Kaggle:

In [ ]:
!python3 /usr/local/bin/kaggle competitions submit -m $URN -c uos-com2028-21-22-cw -f predictions.csv

100% 380k/380k [00:02<00:00, 137kB/s]
Successfully submitted to COM2028 21/22 CW